<div style="text-align: center;">
  <h1><span style="color:blue; font-family:'Bodoni MT Black';">Web Scraping </span><span style="color:red; font-family:'Bodoni MT Black';">API </span><span style="color:black; font-family:'Bodoni MT Black';">Fotos de los Rovers en Marte</span></h1>
</div> 

![Marte|50](https://static.nationalgeographicla.com/files/styles/image_3200/public/02_cruisestageseparation_10k_perseverance.jpg?w=600 )







Esta API está diseñada para recopilar datos de imágenes recopilados por los rovers **Curiosity**, **Opportunity** y **Spirit** de la NASA en Marte y hacerlos más fácilmente disponibles para otros desarrolladores, educadores y científicos ciudadanos. Esta API es mantenida por **Chris Cerami**.

Cada rover tiene su propio conjunto de fotos almacenadas en la base de datos, que se pueden consultar por separado. Existen varias consultas posibles que se pueden realizar a través de la API. 
Se pondra principal enfasis en el filtrado por fecha terresre, los resultados también se pueden filtrar por la cámara con la que se tomó la foto. 

Cada cámara tiene una función y perspectiva únicas, y se nombran de la siguiente manera: 


## Cámaras de los Rovers
| Abreviación | Cámara                                         | Curiosity | Opportunity | Spirit |
|--------------|-------------------------------------------------|------------|-------------|--------|
| FHAZ         | Front Hazard Avoidance Camera                    | ✔          | ✔           | ✔      |
| RHAZ         | Rear Hazard Avoidance Camera                     | ✔          | ✔           | ✔      |
| MAST         | Mast Camera                                     | ✔          |             |        |
| CHEMCAM      | Chemistry and Camera Complex                     | ✔          |             |        |
| MAHLI        | Mars Hand Lens Imager                           | ✔          |             |        |
| MARDI        | Mars Descent Imager                             | ✔          |             |        |
| NAVCAM       | Navigation Camera                               | ✔          | ✔           | ✔      |
| PANCAM       | Panoramic Camera                                |            | ✔           | ✔      |
| MINITES      | Miniature Thermal Emission Spectrometer (Mini-TES)|            | ✔           | ✔      |



## Consulta por fecha terrestre

| Parámetro   | Tipo         | Valor predeterminado | Descripción                                                |
|-------------|--------------|----------------------|------------------------------------------------------------|
| earth_date  | YYYY-MM-DD   | ninguno              | fecha correspondiente en la Tierra para el sol proporcionado |
| camera      | string       | todos                | ver tabla anterior para las abreviaturas                    |
| api_key     | string       | DEMO_KEY             | clave de api.nasa.gov para uso ampliado                     |


### Inicio
1. Lo primero que se necesitara para trabajar con la API sera lo siguiente:

    1.1 Para obtener el API registrate en el siguiente enlace :[APINASA](https://api.nasa.gov/)
    
    1.2 Al registrarte obtendras un codigo en tu correo, el cual vendra a ser el `api_key`.

```python
api_key = "TU_API"
api_url = 'https://api.nasa.gov/mars-photos/api/v1/rovers/'
```
2. El uso de esta API se dividira en 4 partes con distinto enfoque.

In [3]:
api_key = "1I2iukqST7jLAVR2im0mHRT7MaF0TCEB8MonD1zu"
api_url = 'https://api.nasa.gov/mars-photos/api/v1/rovers/'

## 1. Obteniendo el manifiesto de la misión 
- Este manifiesto enumerará los detalles de la misión del Rover para ayudar a reducir las consultas de fotos a la API. 
- El resultado obtenido aparecera en una tabla.

**Codigo:**
```python
import requests
import csv

rovers = ['Curiosity', 'Opportunity', 'Spirit']

def obtener_manifiesto_mision(rover_name):
    url = f'https://api.nasa.gov/mars-photos/api/v1/manifests/{rover_name}?api_key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        manifest = data['photo_manifest']
        
        # Accede a los diferentes datos del manifiesto
        name = manifest['name']
        landing_date = manifest['landing_date']
        launch_date = manifest['launch_date']
        total_photos = manifest['total_photos']
        max_date = manifest['max_date']
        status = manifest['status']
        
        # Retorna los datos del manifiesto como una lista
        return [name, landing_date, launch_date, total_photos, max_date, status]
        
    else:
        print(f"Error al obtener el manifiesto de la misión del rover {rover_name}.")
        return None

# Abre el archivo CSV en modo escritura
with open('manifiesto.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Escribe la fila de encabezado
    writer.writerow(['Nombre', 'Fecha de Aterrizaje', 'Fecha de Lanzamiento', 'Total de Fotos', 'Ulima foto', 'Estado'])
    
    for rover in rovers:
        manifiesto_data = obtener_manifiesto_mision(rover)
        
        if manifiesto_data is not None:
            writer.writerow(manifiesto_data)

print("Manifiestos de misión de los rovers guardados en el archivo manifiesto.csv.")
```



In [16]:
import pandas as pd

# Lee el archivo CSV con un codec específico
df = pd.read_csv('manifiesto.csv', encoding='latin1')

# Muestra el DataFrame como una tabla
tabla = df.to_string(index=False)
print(tabla)

     Nombre Fecha de Aterrizaje Fecha de Lanzamiento  Total de Fotos Fecha Máxima   Estado
  Curiosity          2012-08-06           2011-11-26          657777   2023-06-17   active
Opportunity          2004-01-25           2003-07-07          198439   2018-06-11 complete
     Spirit          2004-01-04           2003-06-10          124550   2010-03-21 complete


### 2. Mostrar las últimas fotos: 
- Ahora con la tabla obtenida se puede observar que existe una columa llamada **Fecha Máxima** que se refiere a la ultima foto que tomo cada rover, en el caso del **Curiosity**, este variara por que se encuentro activo es decir aun sigue tomando fotos en Marte.
- Para poder visualizar estas fotos las descargaremos y guardaremos en nuestro ordenador.

**Codigo:**

In [27]:
import requests
import os
import json

def descargar_ultimas_imagenes(api_key):
    rovers = ['curiosity', 'opportunity', 'spirit']

    for rover in rovers:
        url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/{rover}/latest_photos?api_key={api_key}"
        response = requests.get(url)

        if response.status_code == 200:
            data = json.loads(response.text)
            latest_photos = data["latest_photos"]

            print(f"Rover: {rover.capitalize()}")
            print("----------------------")

            folder_path = f"{rover}_photos"
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            cameras = set()
            for photo in latest_photos:
                camera = photo["camera"]["full_name"]
                cameras.add(camera)

            for camera in cameras:
                print(f"Cámara: {camera}")

                last_photo = None
                for photo in latest_photos:
                    if photo["camera"]["full_name"] == camera:
                        last_photo = photo

                if last_photo:
                    img_src = last_photo["img_src"]
                    date = last_photo["earth_date"]

                    image_data = requests.get(img_src).content
                    image_name = f"{date}_{camera}.jpg"
                    image_path = os.path.join(folder_path, image_name)

                    with open(image_path, "wb") as image_file:
                        image_file.write(image_data)

                    print("Fecha: ", date)
                    print("Imagen guardada en: ", image_path)
                    print("----------------------------------")

            print()

        else:
            print(f"Error al obtener las imágenes del rover {rover}: {response.status_code}")

# Llamar a la función para descargar las últimas imágenes de los rovers
descargar_ultimas_imagenes(api_key)


Rover: Curiosity
----------------------
Cámara: Chemistry and Camera Complex
Fecha:  2023-06-17
Imagen guardada en:  curiosity_photos\2023-06-17_Chemistry and Camera Complex.jpg
----------------------------------

Rover: Opportunity
----------------------
Cámara: Panoramic Camera
Fecha:  2018-06-11
Imagen guardada en:  opportunity_photos\2018-06-11_Panoramic Camera.jpg
----------------------------------

Rover: Spirit
----------------------
Cámara: Panoramic Camera
Fecha:  2010-03-21
Imagen guardada en:  spirit_photos\2010-03-21_Panoramic Camera.jpg
----------------------------------



### 3. Búsqueda personalizada
- En esta parte se realizará una búsqueda personalizada, es decir, uno puede descargar la imagen por fecha, rover o cámara que desee.
- A su vez este se almacenara en una carpeta(ya antes creada en el punto 2) por rover y fecha que contenga la fecha correspondiente de la búsqueda.

In [26]:
import requests
import json
import os
import urllib

def buscar_imagenes(api_key, rover, fecha, camara):
    url = f"https://api.nasa.gov/mars-photos/api/v1/rovers/{rover}/photos?earth_date={fecha}&camera={camara}&api_key={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = json.loads(response.text)
        photos = data["photos"]
        if len(photos) > 0:
            for photo in photos:
                img_src = photo["img_src"]
                date = photo["earth_date"]
                camera = photo["camera"]["full_name"]

                print("Rover: ", rover)
                print("Fecha: ", date)
                print("Cámara: ", camera)
                print("Imagen: ", img_src)
                print("----------------------------------")

                # Crear la carpeta correspondiente al rover si no existe
                if not os.path.exists(f"{rover}_photos"):
                    os.makedirs(f"{rover}_photos")

                # Obtener el nombre del archivo de la imagen
                file_name = img_src.split("/")[-1]

                # Obtener la carpeta específica para la fecha
                folder_path = f"{rover}_photos/{fecha}"
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)

                # Descargar y guardar la imagen en la carpeta del rover y fecha correspondiente
                urllib.request.urlretrieve(img_src, f"{folder_path}/{file_name}")
                print("Imagen guardada correctamente.")

        else:
            print("No hay imágenes disponibles para la fecha y cámara especificadas.")

    else:
        print("Error al obtener las imágenes:", response.status_code)

api_key = "1I2iukqST7jLAVR2im0mHRT7MaF0TCEB8MonD1zu"
rover = 'opportunity'  # Puedes seleccionar 'curiosity', 'opportunity' o 'spirit'
fecha = '2017-01-05'
camara = 'NAVCAM'

buscar_imagenes(api_key, rover, fecha, camara)


Rover:  opportunity
Fecha:  2017-01-05
Cámara:  Navigation Camera
Imagen:  http://mars.nasa.gov/mer/gallery/all/1/n/4603/1N536825391EFFCUBVP1827L0M1-BR.JPG
----------------------------------
Imagen guardada correctamente.
Rover:  opportunity
Fecha:  2017-01-05
Cámara:  Navigation Camera
Imagen:  http://mars.nasa.gov/mer/gallery/all/1/n/4603/1N536825391EFFCUBVP1827R0M1-BR.JPG
----------------------------------
Imagen guardada correctamente.
